In [2]:
!pip install PyQt5

  Obtaining dependency information for PyQt5 from https://files.pythonhosted.org/packages/ca/ac/596e8ca16fd0634542d874c0d79219fc527ea7de73a5000092f60ecbf6e9/PyQt5-5.15.10-cp37-abi3-win_amd64.whl.metadata
  Obtaining dependency information for PyQt5-sip<13,>=12.13 from https://files.pythonhosted.org/packages/8e/e2/d296cb17bae19ba49137a2649934a70d5f48cc99b7daa6ce9cf1aecddfeb/PyQt5_sip-12.13.0-cp311-cp311-win_amd64.whl.metadata
  Using cached PyQt5_Qt5-5.15.2-py3-none-win_amd64.whl (50.1 MB)
Using cached PyQt5-5.15.10-cp37-abi3-win_amd64.whl (6.8 MB)
Using cached PyQt5_sip-12.13.0-cp311-cp311-win_amd64.whl (78 kB)



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib qt
import seaborn as sns
import os
from os import path
import datetime
import xlrd
plt.style.use('seaborn-v0_8-ticks')
from scipy.signal import find_peaks
import itertools
from pylab import *
import csv

Pobieranie zestawu danych

In [ ]:
folder_path_15 = "C:\\Users\\48503\\Desktop\\joint_symbolic\\controlled_breathing"
folders_15 = os.listdir(folder_path_15)
breathing_dict_15 = {folder : {} for folder in folders_15}

for folder_name in folders_15:
    path = os.path.join(folder_path_15, folder_name)
    for file_name in os.listdir(path):
        if file_name.endswith('.csv'):
         df_name = file_name.split('_ODD')[0]
         df = pd.read_csv(os.path.join(path, file_name))
         breathing_dict_15[folder_name][df_name] = df

Pobranie zestawu danych, gdzie występują dwa typy plików csv

In [3]:
folder_path_56 = "C:\\Users\\48503\\Desktop\\joint_symbolic\\controlled_breathing_full2"
folders_56 = os.listdir(folder_path_56)
breathing_dict_56 = {folder : {} for folder in folders_56}

for folder_name in folders_56:
    folder_path = os.path.join(folder_path_56, folder_name)
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
          df_name = file_name.split('PAC_')[1].split('_')[0] #id to numer pacjenta pobrany z nazwy pliku
          file_path = os.path.join(folder_path, file_name)
          csv_file = open(file_path, 'r')
          try:
              dialect = csv.Sniffer().sniff(csv_file.readline(), ';')   
          except:
              dialect = csv.Sniffer().sniff(csv_file.readline(), ',')   
          csv_file.seek(0)
          df = pd.read_csv(file_path, on_bad_lines='skip', sep=dialect.delimiter)
          if dialect.delimiter == ";":
              for row in df:
                  df[row] = df[row].str.replace(',','.').astype(float)
          breathing_dict_56[folder_name][df_name] = df

Pobranie danych z bazy 37 plików, gdzie id pacjenta to numer z nazwy folderu "ochotnik"; można też pobrać id jako numer z nazwy folderu + 56 w przypadku integracji obu baz (df_name = str(int(folder_name.split('OCHOTNIK_')[1].split('_')[0]) + 56)), żeby numery id się nie powtarzały

In [68]:
folder_path_37 = "C:\\Users\\48503\\Desktop\\joint_symbolic\\CONTROL_BREATHING_RECORDINGS"
breathing_dict_keys = ['6_breaths', '10_breaths', '15_breaths', 'baseline']
breathing_dict_37 = {frequency:{} for frequency in breathing_dict_keys}
folders_37 = os.listdir(folder_path_37)

for folder_name in folders_37:
    folder_path = os.path.join(folder_path_37, folder_name)
    df_number = folder_name.split('OCHOTNIK_')[1].split('_')[0] 
    for next_folder_name in os.listdir(folder_path):
        try:
           breathing_frequency = next_folder_name.split('CLEAN_DATA')[1].lower()[1:]
        except:
            breathing_frequency = "RAW_DATA"
        if breathing_frequency in breathing_dict_37:
            next_folder_path = os.path.join(folder_path, next_folder_name)
            for file_name in os.listdir(next_folder_path):
                if file_name.endswith('.csv'): 
                    file_path = os.path.join(next_folder_path, file_name)
                    csv_file = open(file_path, 'r')
                    try:
                        dialect = csv.Sniffer().sniff(csv_file.readline(), ';')   
                    except:
                        dialect = csv.Sniffer().sniff(csv_file.readline(), ',')      
                    csv_file.seek(0)
                    decimal_map = {";" : ",", "," : "."}
                    my_df = pd.read_csv(file_path, encoding='latin-1', decimal=decimal_map[dialect.delimiter], on_bad_lines='skip', sep=dialect.delimiter)
                    
                    breathing_dict_37[breathing_frequency][df_number] = my_df

C:\Users\48503\AppData\Local\Temp\ipykernel_18620\1392453576.py:26: DtypeWarning: Columns (3,13) have mixed types. Specify dtype option on import or set low_memory=False.
  my_df = pd.read_csv(file_path, encoding='latin-1', decimal=decimal_map[dialect.delimiter], on_bad_lines='skip', sep=dialect.delimiter)


Połączenie bazy 37 i 56 plików w jedną

Sprawdzenie nazw kolumn w plikach

In [11]:
def check_column_names(breathing_dict):
    for breath_frequency in breathing_dict:
        for patient_nr, patient_df in breathing_dict[breath_frequency].items(): 
            print(patient_df.columns)                

In [ ]:
check_column_names(breathing_dict_56)

In [12]:
check_column_names(breathing_dict_37)

Index(['DateTime', 'abp_cnap[mmHg]', 'etco2[mmHg]', 'co2[mmHg]', 'rr[rpm]',
       'hbo_s1_d1[umol/L]', 'hbo_s1_d2[umol/L]', 'hbo_s2_d3[umol/L]',
       'hbo_s2_d4[umol/L]', 'hbo_s3_d5[umol/L]', 'hbo_s4_d5[umol/L]',
       'hbo_s4_d6[umol/L]', 'hbo_s5_d6[umol/L]', 'hbr_s1_d1[umol/L]',
       'hbr_s1_d2[umol/L]', 'hbr_s2_d3[umol/L]', 'hbr_s2_d4[umol/L]',
       'hbr_s3_d5[umol/L]', 'hbr_s4_d5[umol/L]', 'hbr_s4_d6[umol/L]',
       'hbr_s5_d6[umol/L]', 'fvl', 'fvr'],
      dtype='object')
Index(['Unnamed: 0', 'time', 'raw_signal', 'cleaned_signal', 'interpolated'], dtype='object')
Index(['DateTime', 'abp_cnap[mmHg]', 'etco2[mmHg]', 'co2[mmHg]', 'rr[rpm]',
       'hbo_s1_d1[umol/L]', 'hbo_s1_d2[umol/L]', 'hbo_s2_d3[umol/L]',
       'hbo_s2_d4[umol/L]', 'hbo_s3_d5[umol/L]', 'hbo_s4_d5[umol/L]',
       'hbo_s4_d6[umol/L]', 'hbo_s5_d6[umol/L]', 'hbr_s1_d1[umol/L]',
       'hbr_s1_d2[umol/L]', 'hbr_s2_d3[umol/L]', 'hbr_s2_d4[umol/L]',
       'hbr_s3_d5[umol/L]', 'hbr_s4_d5[umol/L]', 'hbr_s4_d6

In [13]:
def how_many_have_ecg(breathing_dict):
    for breath_frequency in breathing_dict:
        ecg_number = 0
        patients_with_ecg = []
        for patient_nr, patient_df in breathing_dict[breath_frequency].items(): 
            if "ecg" in patient_df.columns:
                ecg_number += 1
                patients_with_ecg.append(patient_nr)
        print(f"Liczba pacjentów z ECG dla częstotliwości {breath_frequency}: {ecg_number}")  
        print(f"Numery pacjentów, którzy mają ECG dla częstotliwości {breath_frequency}: {patients_with_ecg} \n")

In [14]:
how_many_have_ecg(breathing_dict_37)

Liczba pacjentów z ECG dla częstotliwości 6_breaths: 17
Numery pacjentów, którzy mają ECG dla częstotliwości 6_breaths: ['21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37'] 

Liczba pacjentów z ECG dla częstotliwości 10_breaths: 17
Numery pacjentów, którzy mają ECG dla częstotliwości 10_breaths: ['21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37'] 

Liczba pacjentów z ECG dla częstotliwości 15_breaths: 17
Numery pacjentów, którzy mają ECG dla częstotliwości 15_breaths: ['21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37'] 

Liczba pacjentów z ECG dla częstotliwości baseline: 18
Numery pacjentów, którzy mają ECG dla częstotliwości baseline: ['20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37'] 


Zamiana nazw kolumn na jednolite
"fv_r[]", "fv_r[fv_r]" -> "fv_r"
"fv_l[]", "fv_r[fv_l]" -> "fv_l"
"ekg[]", "ekg[ekg]" -> "ekg"
"abp_arm[mm_Hg]", "abp_arm[abp_arm]" -> "abp_arm"

In [49]:
def unify_column_names(breathing_dict):
    column_names_map = {"fv_r[]":"fv_r", "fv_r[fv_r]":"fv_r", "fvr":"fv_r", "fv_l[]":"fv_l", 
                    "fv_l[fv_l]":"fv_l", "fvl":"fv_l", "ekg[]":"ekg", "ekg[ekg]":"ekg", "ecg":"ekg", "abp_arm[mm_Hg]":"abp_arm", "abp_arm[abp_arm]":"abp_arm", "abp_cnap[mmHg]":"abp_arm", "etco2[mmHg]":"etco2"}
    for breath_frequency in breathing_dict:
        for patient_nr, patient_df in breathing_dict[breath_frequency].items(): 
            patient_df.rename(columns=column_names_map, inplace=True)

In [50]:
unify_column_names(breathing_dict_37)

Sprawdzenie nazw kolumn w plikach po raz drugi

In [ ]:
check_column_names(breathing_dict_56)

In [17]:
check_column_names(breathing_dict_37)

Index(['DateTime', 'abp_arm', 'etco2', 'co2[mmHg]', 'rr[rpm]',
       'hbo_s1_d1[umol/L]', 'hbo_s1_d2[umol/L]', 'hbo_s2_d3[umol/L]',
       'hbo_s2_d4[umol/L]', 'hbo_s3_d5[umol/L]', 'hbo_s4_d5[umol/L]',
       'hbo_s4_d6[umol/L]', 'hbo_s5_d6[umol/L]', 'hbr_s1_d1[umol/L]',
       'hbr_s1_d2[umol/L]', 'hbr_s2_d3[umol/L]', 'hbr_s2_d4[umol/L]',
       'hbr_s3_d5[umol/L]', 'hbr_s4_d5[umol/L]', 'hbr_s4_d6[umol/L]',
       'hbr_s5_d6[umol/L]', 'fv_l', 'fv_r'],
      dtype='object')
Index(['Unnamed: 0', 'time', 'raw_signal', 'cleaned_signal', 'interpolated'], dtype='object')
Index(['DateTime', 'abp_arm', 'etco2', 'co2[mmHg]', 'rr[rpm]',
       'hbo_s1_d1[umol/L]', 'hbo_s1_d2[umol/L]', 'hbo_s2_d3[umol/L]',
       'hbo_s2_d4[umol/L]', 'hbo_s3_d5[umol/L]', 'hbo_s4_d5[umol/L]',
       'hbo_s4_d6[umol/L]', 'hbo_s5_d6[umol/L]', 'hbr_s1_d1[umol/L]',
       'hbr_s1_d2[umol/L]', 'hbr_s2_d3[umol/L]', 'hbr_s2_d4[umol/L]',
       'hbr_s3_d5[umol/L]', 'hbr_s4_d5[umol/L]', 'hbr_s4_d6[umol/L]',
       'hbr_s

Funkcja konwertująca czas

In [51]:
def icmp_dateformat_to_datetime(icmp_time_mark):
    datetime_date = xlrd.xldate_as_datetime(icmp_time_mark, 0)
    datetime_date = datetime_date + datetime.timedelta(hours=1)
    return datetime_date

Sprawdzenie procentu brakujących wartości w kolumnach z ecg, abp, fv oraz etco2

In [33]:
def print_nan_percent(breathing_dict):
    nan_dict = {breath_frequency : {} for breath_frequency in breathing_dict}
    for breath_frequency in breathing_dict:
        for patient_nr, patient_df in breathing_dict[breath_frequency].items():
            nan_string = ""
            for col_name in ['abp_arm', 'ekg', 'fv_r', 'fv_l', 'etco2']:
                if col_name in patient_df.columns:
                    nan_percent = patient_df[col_name].isnull().sum() * 100 / len(patient_df)
                    if nan_percent != 0.0:
                       nan_string += col_name + " : " + str(round(nan_percent, 2)) + "; "
            nan_dict[breath_frequency][patient_nr] = nan_string
    nan_df = pd.DataFrame(nan_dict)
    return nan_df

In [ ]:
print_nan_percent(breathing_dict_56)

In [34]:
print_nan_percent(breathing_dict_37)

,6_breaths,10_breaths,15_breaths,baseline
1,,,,
10,,NaN,,
12,,,,
13,,,,
14,,fv_r : 0.13; fv_l : 0.13; etco2 : 0.2;,,
15,etco2 : 21.43;,etco2 : 52.85;,,
16,,,,
17,,,,
18,,,,
19,,,,


In [52]:
def delete_keys(to_delete, breathing_dict):
    for breath_freq in breathing_dict:
        for key_to_delete in to_delete:
            breathing_dict[breath_freq].pop(key_to_delete, None)

Usuwanie ochotnika numer 35 z bazy 37 (zła jakość sygnału) oraz ochotnika numer 10 (jeszcze nie ma pliku do 10 oddechów oraz tylko fv do 6 i 15 oddechów)

In [53]:
to_delete_37 = ["35", "10"]
delete_keys(to_delete_37, breathing_dict_37)

Dodanie kolumny z datą i godziną do każdego dataframe i wypełnienie braków danych (NaN) wartościami średnimi

In [54]:
def fill_nan_and_add_time(breathing_dict):    
    for breathing_freq in breathing_dict:
        for patient_number, patient_df in breathing_dict[breathing_freq].items():
            print(patient_number +": "+breathing_freq)
            patient_df['DateFormat'] = patient_df['DateTime'].map(icmp_dateformat_to_datetime)
            patient_df.fillna(patient_df.mean(), inplace=True)

In [ ]:
fill_nan_and_add_time(breathing_dict_56)

In [44]:
fill_nan_and_add_time(breathing_dict_37)

1: 6_breaths
12: 6_breaths
13: 6_breaths
14: 6_breaths
15: 6_breaths
16: 6_breaths
17: 6_breaths
18: 6_breaths
19: 6_breaths
2: 6_breaths
20: 6_breaths
21: 6_breaths
22: 6_breaths
23: 6_breaths
24: 6_breaths
25: 6_breaths
26: 6_breaths
27: 6_breaths
28: 6_breaths
29: 6_breaths
3: 6_breaths
30: 6_breaths
31: 6_breaths
32: 6_breaths
33: 6_breaths
34: 6_breaths
36: 6_breaths
37: 6_breaths
4: 6_breaths
5: 6_breaths
6: 6_breaths
7: 6_breaths
8: 6_breaths
9: 6_breaths
1: 10_breaths
12: 10_breaths
13: 10_breaths
14: 10_breaths
15: 10_breaths
16: 10_breaths
17: 10_breaths
18: 10_breaths
19: 10_breaths
2: 10_breaths
20: 10_breaths
21: 10_breaths
22: 10_breaths
23: 10_breaths
24: 10_breaths
25: 10_breaths
26: 10_breaths
27: 10_breaths
28: 10_breaths
29: 10_breaths
3: 10_breaths


TypeError: '<' not supported between instances of 'str' and 'int'

In [63]:
breathing_dict_37["6_breaths"]["3"]

KeyError: '3'

Funkcja znajdująca piki, diff to jest RR w przypadku ECG tylko trzeba je przemnożyć przez 0.005 (1/(200 Hz)), peaks to numery próbek odpowiadających pikom, signal[peaks] to wartości sygnału odpowiadające tym numerom próbek

In [93]:
def get_peaks(signal):
    peaks, _ = find_peaks(signal, distance=115)
    difference = np.diff(peaks)  
    return peaks, signal[peaks], difference

Wyrysowanie 600 pierwszych próbek każdego sygnału z zaznaczonymi znalezionymi pikami ("x")

In [95]:
def check_peak_detection_accuracy(plot_folder_path, data_dict):
    for breath_frequency in data_dict:
        for dataframe_nr, dataframe in data_dict[breath_frequency].items():       
            fig, ax = plt.subplots(4, 1, tight_layout = True, figsize=(16, 32)) 
            for i, col_name in enumerate(['abp_arm', 'ekg', 'fv_l', 'fv_r']):
                if col_name in dataframe.columns:
                    x = dataframe[col_name].iloc[:600]
                    ax[i].plot(x)
                    my_peaks = get_peaks(x)
                    ax[i].plot(my_peaks[0], my_peaks[1], "x")
                    ax[i].set_ylabel(f'{col_name}', fontsize=25)
                    ax[i].tick_params(axis='both', labelsize=18)
                    ax[i].grid(True)
            ax[3].set_xlabel("nr probki", fontsize = 28)
            savefig_path = plot_folder_path+f"\\{breath_frequency}\\{dataframe_nr}_plot.pdf"
            plt.savefig(savefig_path, bbox_inches='tight', dpi=600)
            plt.show()
            plt.close()

In [15]:
folder_path = "C:\\Users\\48503\\Desktop\\joint_symbolic\\timeseries_plots\\56"
check_peak_detection_accuracy(folder_path, breathing_dict_56)

In [97]:
folder_path = "C:\\Users\\48503\\Desktop\\joint_symbolic\\timeseries_plots\\37"
check_peak_detection_accuracy(folder_path, breathing_dict_37)

In [98]:
breathing_dict_37["baseline"]["28"]


,DateTime,abp_arm,ekg,etco2[mmHg],co2[mmHg],rr[rpm],hbo_s1_d1[umol/L],hbo_s1_d2[umol/L],hbo_s2_d3[umol/L],hbo_s2_d4[umol/L],...,hbr_s1_d1[umol/L],hbr_s1_d2[umol/L],hbr_s2_d3[umol/L],hbr_s2_d4[umol/L],hbr_s3_d5[umol/L],hbr_s4_d5[umol/L],hbr_s4_d6[umol/L],hbr_s5_d6[umol/L],fv_l,fv_r
0,45279.549604,70.31,0.0049,41.99,35.990,9.989,-0.9960,-1.1070,-0.6005,-0.1196,...,-0.0513,-0.0023,-0.0053,0.5242,-0.5110,-0.2965,-0.1982,-0.2545,-0.0092,50.38
1,45279.549604,69.34,0.0244,41.99,35.990,9.989,-0.9960,-1.1070,-0.6005,-0.1196,...,-0.0513,-0.0023,-0.0053,0.5242,-0.5110,-0.2965,-0.1982,-0.2545,-0.0092,52.73
2,45279.549604,68.85,0.0049,41.99,35.990,9.989,-0.9810,-1.1050,-0.5956,-0.1080,...,-0.0600,-0.0041,-0.0110,0.5109,-0.5135,-0.2878,-0.2019,-0.2593,-0.0092,52.73
3,45279.549604,68.85,0.0049,41.99,35.990,9.989,-0.9810,-1.1050,-0.5956,-0.1080,...,-0.0600,-0.0041,-0.0110,0.5109,-0.5135,-0.2878,-0.2019,-0.2593,-0.0092,55.07
4,45279.549604,68.85,0.0244,41.99,35.990,9.989,-0.9810,-1.1050,-0.5956,-0.1080,...,-0.0600,-0.0041,-0.0110,0.5109,-0.5135,-0.2878,-0.2019,-0.2593,-0.0092,56.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59174,45279.553028,73.24,0.0049,35.99,5.988,12.990,-0.1909,-0.1402,-0.1849,0.0990,...,-0.0346,-0.0375,0.0053,0.2449,0.0549,0.0623,0.0214,-0.0812,-0.0092,42.18
59175,45279.553028,73.24,0.0049,35.99,5.988,12.990,-0.1909,-0.1402,-0.1849,0.0990,...,-0.0346,-0.0375,0.0053,0.2449,0.0549,0.0623,0.0214,-0.0812,-0.0092,42.18
59176,45279.553028,74.22,0.0049,35.99,5.988,12.990,-0.1909,-0.1402,-0.1849,0.0990,...,-0.0346,-0.0375,0.0053,0.2449,0.0549,0.0623,0.0214,-0.0812,-0.0092,42.18
59177,45279.553028,74.71,0.0195,35.99,5.988,12.990,-0.2213,-0.1494,-0.1813,0.1028,...,-0.0122,-0.0334,-0.0041,0.2409,0.0568,0.0586,0.0211,-0.0859,-0.0092,42.18


Usuwanie uprzednio wykluczonych pacjentów numer 2, 6 i 33 oraz dodatkowo 14 i 31 (zła jakość sygnału) z bazy 56

In [89]:
to_delete_56 = ["2", "6", "33", "14", "31"]
delete_keys(to_delete_56, breathing_dict_56)

Kopie słownika z danymi do wykonania JSA dla ABP i FV, wszystkie osoby mają analizę dla ABP (w przypadku 19 osób zamiast EKG używany jest sygnał ABP do obliczenia RR), jedna osoba nie ma wykonywanej analizy dla FV

In [161]:
breathing_dict_37_ABP = breathing_dict_37.copy()

In [106]:
breathing_dict_37_FV = breathing_dict_37.copy()
to_delete_37_FV = ["28"]
delete_keys(to_delete_37_FV, breathing_dict_37_FV)

In [107]:
def make_empty_matrix():
    my_permutations = [''.join(map(str, triplet)) for triplet in list((itertools.product([1, 0], repeat=3)))]
    my_matrix = pd.DataFrame()
    for triplet_1 in my_permutations: 
       for triplet_2 in my_permutations:
            my_matrix.at["'"+triplet_1, "'"+triplet_2] = 0
    return my_matrix

In [123]:
def get_diff_df(dataframe, is_fv, is_left):
    sap = get_peaks(dataframe['abp_arm'])[1]
    if is_fv:
        sap = get_peaks(dataframe['fv_l'])[1]
        if not is_left:
            sap = get_peaks(dataframe['fv_r'])[1]
            
    try:        
        rr = get_peaks(dataframe['ekg'])[2]*0.005
    except:
        rr = get_peaks(dataframe['abp_arm'])[2]*0.005 #u osób bez EKG korzystamy z ABP
        
    sap_diff = np.diff(sap)
    rr_diff = np.diff(rr)
    diff_dataframe = pd.DataFrame(list(zip(sap_diff, rr_diff)), columns=['sap_diff', 'rr_diff'])
    return diff_dataframe

In [135]:
def make_and_save_heatmap(relative_array, breath_frequency, is_fv, heatmap_path):
    my_custom_palette = sns.color_palette("ch:-.24", as_cmap=True)
    my_heatmap = sns.heatmap(relative_array, cmap = my_custom_palette, annot=True)
    my_heatmap.xaxis.tick_top()
    my_heatmap.xaxis.set_ticks_position('none')
    fontsize = 10
    plt.xlabel('SAP', fontsize = fontsize) 
    my_heatmap.xaxis.set_label_position('top') 
    plt.ylabel('RR', fontsize = fontsize) 
    if is_fv:
        plt.savefig(heatmap_path+f"\\{breath_frequency}_heatmap_fv.pdf", bbox_inches='tight', dpi=600)
        relative_array.to_csv(heatmap_path+f"\\{breath_frequency}_matrix_fv.csv")
    else:
        plt.savefig(heatmap_path+f"\\{breath_frequency}_heatmap_abp.pdf", bbox_inches='tight', dpi=600)
        relative_array.to_csv(heatmap_path+f"\\{breath_frequency}_matrix_abp.csv")
    plt.show()
    plt.close()

In [125]:
def save_jsd(is_fv, jsd_df, jsd_path):
    if is_fv:
        jsd_df.T.to_csv(jsd_path+"\\jsd_fv.csv")
    else:    
        jsd_df.T.to_csv(jsd_path+"\\jsd_abp.csv")

In [133]:
def joint_symbolic(data_dict, is_fv, metadata_path, heatmap_path, jsd_path):
    jsd_df = pd.DataFrame()
    metadata = pd.read_csv(metadata_path, sep = ";")
    for freq in data_dict:
        freq_matrix = make_empty_matrix()
        for patient_number, dataframe in data_dict[freq].items():
            is_left = True
            if is_fv:
                is_left = bool(metadata.at[metadata.loc[metadata['L.poj.'] == int(patient_number)].index[0], 'FV_LEWA_MCA'])
            patient_freq_matrix = make_empty_matrix()
            diff_df = get_diff_df(dataframe, is_fv, is_left)
            
            for col_name in ['sap_diff', 'rr_diff']:
                diff_df[f'{col_name}_binary'] = diff_df.apply(lambda df_row: 1 if df_row[col_name] > 0 else 0, axis = 1)
            
            for index in range(0, len(diff_df)):
                sap_triplet =  ''.join(diff_df['sap_diff_binary'].iloc[index:index+3].astype(str))
                rr_triplet =  ''.join(diff_df['rr_diff_binary'].iloc[index:index+3].astype(str))
                if len(rr_triplet) > 2 and len(sap_triplet) > 2:
                     freq_matrix.at["'"+rr_triplet, "'"+sap_triplet] += 1 #kreska potrzebna, żeby R nie usuwało początkowych zer
                     patient_freq_matrix.at["'"+rr_triplet, "'"+sap_triplet] += 1 
            
            relative_patient_matrix = patient_freq_matrix.div(np.sum(patient_freq_matrix.values))   
            signal_map = {True : '_FV', False : '_ABP'}
            jsd_df.at["jsd_sym_"+freq+signal_map[is_fv], patient_number] = np.sum(np.diag(relative_patient_matrix)) 
            jsd_df.at["jsd_diam_"+freq+signal_map[is_fv], patient_number] = np.sum(np.diag(np.rot90(relative_patient_matrix)))
            
        relative_freq_matrix = freq_matrix.div(np.sum(freq_matrix.values))        
        make_and_save_heatmap(relative_freq_matrix, freq, is_fv, heatmap_path)
    save_jsd(is_fv, jsd_df, jsd_path)     

In [126]:
jsa_path = "C:\\Users\\48503\\Desktop\\joint_symbolic\\"

Joint symbolic dla bazy 56

In [120]:
meta_path_56 = jsa_path + "dane_sonata\\dane_sonata_56.csv"
heatmap_path_56 = jsa_path + "heatmaps\\56"
jsd_path_56 = jsa_path + "jsd\\56"

In [98]:
joint_symbolic(breathing_dict_56, False, meta_path_56, heatmap_path_56, jsd_path_56) #abp
joint_symbolic(breathing_dict_56, True, meta_path_56, heatmap_path_56, jsd_path_56)  #fv

Joint symbolic dla bazy 37

In [127]:
meta_path_37 = jsa_path + "dane_sonata\\dane_sonata_37.csv"
heatmap_path_37 = jsa_path + "heatmaps\\37"
jsd_path_37 = jsa_path + "jsd\\37"

In [134]:
joint_symbolic(breathing_dict_37_ABP, False, meta_path_37, heatmap_path_37, jsd_path_37) #abp
joint_symbolic(breathing_dict_37_FV, True, meta_path_37, heatmap_path_37, jsd_path_37)  #fv

Zmiana formy macierzy z relatywną częstotliwością na zwykłą tabelkę w celu narysowania histogramów 3D w R

In [117]:
def transform_matrix(heatmap_path):
    save_path = heatmap_path+"\\transformed"
    for my_file_name in os.listdir(path):
        if my_file_name.endswith('.csv'):
           dataframe = pd.read_csv(path+f"\\{my_file_name}", index_col='Unnamed: 0')
           new_df = pd.DataFrame(columns=['RR', 'SAP', 'relative frequency'])

           for idx in dataframe.index:
              for col in dataframe.columns:
                 new_df = pd.concat([new_df, pd.DataFrame({'RR': [idx], 'SAP': [col], 'relative frequency': [dataframe.at[idx, col]]})], ignore_index=True)
           new_df.to_csv(save_path+"\\"+file_name, index=False)

In [ ]:
transform_matrix(heatmap_path_37)

In [137]:
breathing_dict_37["10_breaths"]["34"]

,DateTime,abp_arm,ekg,etco2[mmHg],co2[mmHg],rr[rpm],hbo_s1_d1[umol/L],hbo_s1_d2[umol/L],hbo_s2_d3[umol/L],hbo_s2_d4[umol/L],...,hbr_s1_d1[umol/L],hbr_s1_d2[umol/L],hbr_s2_d3[umol/L],hbr_s2_d4[umol/L],hbr_s3_d5[umol/L],hbr_s4_d5[umol/L],hbr_s4_d6[umol/L],hbr_s5_d6[umol/L],fv_l,fv_r
0,45294.569349,106.0,-0.0097,39.99,11.99,17.99,0.1251,0.2403,0.0348,0.1211,...,0.0370,0.0122,-0.0114,0.0597,0.0121,-0.0278,-0.1152,0.0995,-0.0094,72.65
1,45294.569349,106.4,-0.0440,39.99,11.99,17.99,0.1425,0.2329,0.0038,0.1112,...,0.0384,-0.0018,0.0032,0.0635,0.0084,-0.0186,-0.1108,0.0978,-0.0094,72.65
2,45294.569349,106.4,-0.0097,39.99,4.99,17.99,0.1425,0.2329,0.0038,0.1112,...,0.0384,-0.0018,0.0032,0.0635,0.0084,-0.0186,-0.1108,0.0978,-0.0094,72.65
3,45294.569349,106.9,0.0391,39.99,4.99,17.99,0.1425,0.2329,0.0038,0.1112,...,0.0384,-0.0018,0.0032,0.0635,0.0084,-0.0186,-0.1108,0.0978,-0.0094,72.65
4,45294.569349,106.4,0.0391,39.99,4.99,17.99,0.1755,0.2118,0.0121,0.0932,...,-0.0115,0.0195,-0.0012,0.0696,0.0055,-0.0326,-0.1050,0.0993,-0.0094,71.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60537,45294.572852,102.1,-0.0244,41.99,39.99,10.99,-0.0072,0.1872,0.0029,0.0508,...,-0.0257,-0.0804,-0.0066,-0.0207,-0.0795,-0.1509,-0.0735,-0.0780,-0.0094,62.10
60538,45294.572852,102.1,0.0244,41.99,39.99,10.99,-0.0072,0.1872,0.0029,0.0508,...,-0.0257,-0.0804,-0.0066,-0.0207,-0.0795,-0.1509,-0.0735,-0.0780,-0.0094,62.10
60539,45294.572852,101.6,0.0830,41.99,39.99,10.99,-0.0072,0.1872,0.0029,0.0508,...,-0.0257,-0.0804,-0.0066,-0.0207,-0.0795,-0.1509,-0.0735,-0.0780,-0.0094,59.76
60540,45294.572852,101.1,0.1464,41.99,39.99,10.99,-0.0177,0.1541,0.0435,0.0351,...,-0.0090,-0.0479,-0.0156,0.0065,-0.0825,-0.1437,-0.0635,-0.0743,-0.0094,57.41
